In [2]:
!pip install transformers

     |████████████████████████████████| 2.5MB 5.4MB/s 
     |████████████████████████████████| 3.3MB 33.8MB/s 
     |████████████████████████████████| 901kB 33.8MB/s 


In [3]:
import torch

from transformers import BertForQuestionAnswering

model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [4]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [5]:
question = "How many parameters does BERT-large have?"
answer_text = "BERT-large is really big... it has 24-layers and an embedding size of 1,024, for a total of 340M parameters! Altogether it is 1.34GB, so expect it to take a couple minutes to download to your Colab instance."

In [6]:
# Apply the tokenizer to the input text, treating them as a text-pair.
input_ids = tokenizer.encode(question, answer_text)

print('The input has a total of {:} tokens.'.format(len(input_ids)))

The input has a total of 70 tokens.


In [7]:
# Search the input_ids for the first instance of the `[SEP]` token.
sep_index = input_ids.index(tokenizer.sep_token_id)

# The number of segment A tokens includes the [SEP] token istelf.
num_seg_a = sep_index + 1

# The remainder are segment B.
num_seg_b = len(input_ids) - num_seg_a

# Construct the list of 0s and 1s.
segment_ids = [0]*num_seg_a + [1]*num_seg_b

# There should be a segment_id for every input token.
assert len(segment_ids) == len(input_ids)

In [8]:
# Run our example through the model.
start_scores, end_scores = model(torch.tensor([input_ids]), # The tokens representing our input text.
                                 token_type_ids=torch.tensor([segment_ids]), return_dict=False) # The segment IDs to differentiate question from answer_text


In [9]:
tokens = tokenizer.convert_ids_to_tokens(input_ids)

In [10]:
# Find the tokens with the highest `start` and `end` scores.
answer_start = torch.argmax(start_scores)
answer_end = torch.argmax(end_scores)

# Start with the first token.
answer = tokens[answer_start]

# Select the remaining answer tokens and join them with whitespace.
for i in range(answer_start + 1, answer_end + 1):
    
    # If it's a subword token, then recombine it with the previous token.
    if tokens[i][0:2] == '##':
        answer += tokens[i][2:]
    
    # Otherwise, add a space then the token.
    else:
        answer += ' ' + tokens[i]

print('Answer: "' + answer + '"')

Answer: "340m"


In [31]:
def answer_question(question, answer_text):
    '''
    Takes a `question` string and an `answer_text` string (which contains the
    answer), and identifies the words within the `answer_text` that are the
    answer. Prints them out.
    '''
    # ======== Tokenize ========
    # Apply the tokenizer to the input text, treating them as a text-pair.
    input_ids = tokenizer.encode(question, answer_text)

    # Report how long the input sequence is.
    print('Query has {:,} tokens.\n'.format(len(input_ids)))

    # ======== Set Segment IDs ========
    # Search the input_ids for the first instance of the `[SEP]` token.
    sep_index = input_ids.index(tokenizer.sep_token_id)

    # The number of segment A tokens includes the [SEP] token istelf.
    num_seg_a = sep_index + 1

    # The remainder are segment B.
    num_seg_b = len(input_ids) - num_seg_a

    # Construct the list of 0s and 1s.
    segment_ids = [0]*num_seg_a + [1]*num_seg_b

    # There should be a segment_id for every input token.
    assert len(segment_ids) == len(input_ids)

    # ======== Evaluate ========
    # Run our example question through the model.
    start_scores, end_scores = model(torch.tensor([input_ids]), # The tokens representing our input text.
                                    token_type_ids=torch.tensor([segment_ids]), return_dict=False) # The segment IDs to differentiate question from answer_text

    # ======== Reconstruct Answer ========
    # Find the tokens with the highest `start` and `end` scores.
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)

    # Get the string versions of the input tokens.
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    answer = full_sentence (tokens, answer_start, answer_end)
    

    print('Answer: "' + answer + '"')

In [83]:
def full_sentence (tokens, answer_start, answer_end):

    # Start with the first token and/or first letter in sentence
    if tokens[answer_start-1] == '.' or tokens[answer_start] == None:
        answer = tokens[answer_start]
    else: 
        while tokens[answer_start] != '.':
            answer_start-=1
    
    # Find the first letter in the sentence
    if tokens[answer_end] != '.' or tokens[answer_end] != None:
        while tokens[answer_end] != '.' or answer_end+1 >= len(tokens):
            answer_end+=1

    answer = tokens[answer_start]

    # Select the remaining answer tokens and join them with whitespace.
    for i in range(answer_start + 1, answer_end + 1):
        
        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]
        
        # Otherwise, add a space then the token.
        elif tokens[i].isalnum() == False:
            answer += tokens[i]
        else:
            answer += ' ' + tokens[i]
    while answer[0].isalnum() == False:
        answer = answer[1:]
    len_answer = len(answer)
    while answer[len_answer-1].isalnum() == False:
        answer = answer[:len_answer-1]
        len_answer-=1
    return answer[0].upper()+answer[1:]+'.'


In [84]:
question = "Can I get a refund?"

answer_question(question, bert_abstract)

Query has 156 tokens.

Answer: "As noted in the youtube paid service terms of service, if an item of purchased content becomes unavailable during the five year period from the purchase date, you may request a refund."


In [15]:
import textwrap

# Wrap text to 80 characters.
wrapper = textwrap.TextWrapper(width=80) 

bert_abstract = "Purchased Content. When you purchase an item of content, your content will be stored in a digital locker and you may view it an unlimited number of times for during your Locker Period. The “Locker Period” will be for at least 5 years from the date of your purchase (subject to the restrictions described in the YouTube Paid Service Terms of Service). Each item of purchased content may have a different Locker Period and you agree to the Locker Period before you order it. Pausing, stopping, or rewinding purchased content will not extend the Locker Period.  As noted in the YouTube Paid Service Terms of Service, if an item of purchased content becomes unavailable during the five year period from the purchase date, you may request a refund."
print(wrapper.fill(bert_abstract))


Purchased Content. When you purchase an item of content, your content will be
stored in a digital locker and you may view it an unlimited number of times for
during your Locker Period. The “Locker Period” will be for at least 5 years from
the date of your purchase (subject to the restrictions described in the YouTube
Paid Service Terms of Service). Each item of purchased content may have a
different Locker Period and you agree to the Locker Period before you order it.
Pausing, stopping, or rewinding purchased content will not extend the Locker
Period.  As noted in the YouTube Paid Service Terms of Service, if an item of
purchased content becomes unavailable during the five year period from the
purchase date, you may request a refund.


In [29]:
question = "Can I get a refund?"

answer_question(question, bert_abstract)

Query has 156 tokens.

tokens answer start: if
Answer: "if an item of purchased content becomes unavailable during the five year period from the purchase date"
